In this example we
- download a source file
- access netCDF grids
- perform a mesh decimation to reduce point numbers

In [ ]:
# Download the data from source
import urllib.request
import tarfile
import os.path

fileout = 'data.tar.gz'

if not os.path.isfile(fileout):
  url='http://onlinelibrary.wiley.com/store/10.1002/2017JB014296/asset/supinfo/2017JB014296-sup-0002-Data_Set_SI-S01_AA.gz?v=1&s=867faff69e418861f269b9e4b4feaf9268154f65'
  print("Downloading source file")
  urllib.request.urlretrieve(url, 'data.tar.gz')
else:
  print("File exists.")

print("Extracting to local folder")
f = tarfile.open(name='data.tar.gz', mode='r').extractall()

In [ ]:
# Access netCDF and create a vtkPoints oject with the data
import netCDF4
import vtk

vexag = 1

colors = vtk.vtkNamedColors()
interface = netCDF4.Dataset("grids/Crust.nc")
nx = interface.variables['x'].shape[0]
ny = interface.variables['y'].shape[0]
points = vtk.vtkPoints()
print("Creating vtkPoints")
for px in range(nx):
    for py in range(ny):
        zval = interface.variables['z'][py, px]*vexag
        if zval > 0:
          zval = 0
        points.InsertNextPoint(interface.variables['x'][px],
                               interface.variables['y'][py],
                               zval)

pointPD = vtk.vtkPolyData()
pointPD.SetPoints(points)

In [ ]:
# Perform delaunay triangulation
delny = vtk.vtkDelaunay2D()
delny.SetInputData(pointPD)
delny.Update()

In [ ]:
# Optional plot of the triangulation
if False:
  aCellArray = vtk.vtkCellArray()
  boundary = vtk.vtkPolyData()
  boundary.SetPoints(pointPD.GetPoints())
  boundary.SetPolys(aCellArray)

  meshMapper = vtk.vtkPolyDataMapper()
  meshMapper.SetInputConnection(delny.GetOutputPort())

  meshActor = vtk.vtkActor()
  meshActor.SetMapper(meshMapper)
  meshActor.GetProperty().SetEdgeColor(0, 0, 1)
  meshActor.GetProperty().SetInterpolationToFlat()
  meshActor.GetProperty().SetRepresentationToWireframe()

  boundaryMapper = vtk.vtkPolyDataMapper()
  if vtk.VTK_MAJOR_VERSION <= 5:
      boundaryMapper.SetInputConnection(boundary.GetProducerPort())
  else:
      boundaryMapper.SetInputData(boundary)

  boundaryActor = vtk.vtkActor()
  boundaryActor.SetMapper(boundaryMapper)
  boundaryActor.GetProperty().SetColor(1, 0, 0)

  renderer = vtk.vtkRenderer()
  renderWindow = vtk.vtkRenderWindow()
  renderWindow.AddRenderer(renderer)
  renderWindowInteractor = vtk.vtkRenderWindowInteractor()
  renderWindowInteractor.SetRenderWindow(renderWindow)
  renderer.AddActor(meshActor)
  renderer.AddActor(boundaryActor)
  renderer.SetBackground(.3, .6, .3)

  renderWindowInteractor.Initialize()
  renderWindow.Render()
  renderWindowInteractor.Start()

In [ ]:
# Decimate triangulation
print("Decimate")
AbsoluteError=1. # Don't know if that works?!
print("Absolute error:", AbsoluteError)
deci = vtk.vtkDecimatePro()
deci.SetInputConnection(delny.GetOutputPort())
deci.SetErrorIsAbsolute(1) # Read from SetAbsoluteError
#deci.SetAbsoluteError(AbsoluteError) # What is the difference to max error?
deci.SetMaximumError(AbsoluteError)
deci.SetTargetReduction(0.9)
deci.PreserveTopologyOn()
deci.SetFeatureAngle(19)
deci.Update()

In [ ]:
# Save decimated points to file
import numpy as np
decimatedPoly = vtk.vtkPolyData()
decimatedPoly.ShallowCopy(deci.GetOutput())
for i in range(decimatedPoly.GetNumberOfPoints()):
    if i is 0:
        final_points = np.array(decimatedPoly.GetPoint(i))
    else:
        final_points = np.vstack((final_points, decimatedPoly.GetPoint(i)))
np.savetxt('DecimatedCrust.csv', final_points, fmt='%f', delimiter=',')

In [ ]:
# Create point plot
final_points.shape
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.scatter(final_points[:,0], final_points[:,1], c=final_points[:,2])
plt.show()